In [47]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime

#Model Selection/Assessment Packages
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import classification_report, precision_recall_curve, confusion_matrix, auc

In [48]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

Success: downloaded bank_clean.csv.


In [49]:
df = pd.read_csv('bank_clean.csv')
df.info()
train = df
# train['job_student'] = ((train['job_student'] - train['job_student'].mean())/(train['job_student'].std()))
# train['job_technician'] = ((train['job_technician'] - train['job_technician'].mean())/(train['job_technician'].std()))
# train['job_unemployed'] = ((train['job_unemployed'] - train['job_unemployed'].mean())/(train['job_unemployed'].std()))
# train['job_unknown'] = ((train['job_unknown'] - train['job_unknown'].mean())/(train['job_unknown'].std()))
# train['marital_divorced'] = ((train['marital_divorced'] - train['marital_divorced'].mean())/(train['marital_divorced'].std()))
# train['marital_single'] = ((train['marital_single'] - train['marital_single'].mean())/(train['marital_single'].std()))
# train['marital_unknown'] = ((train['marital_unknown'] - train['marital_unknown'].mean())/(train['marital_unknown'].std()))
# train['education_basic.4y'] = ((train['education_basic.4y'] - train['education_basic.4y'].mean())/(train['education_basic.4y'].std()))
# train['education_basic.6y'] = ((train['education_basic.6y'] - train['education_basic.6y'].mean())/(train['education_basic.6y'].std()))
# train['education_basic.9y'] = ((train['education_basic.9y'] - train['education_basic.9y'].mean())/(train['education_basic.9y'].std()))
# train['education_high.school'] = ((train['education_high.school'] - train['education_high.school'].mean())/(train['education_high.school'].std()))
# train['education_illiterate'] = ((train['education_illiterate'] - train['education_illiterate'].mean())/(train['education_illiterate'].std()))
# train['education_professional.course'] = ((train['education_professional.course'] - train['education_professional.course'].mean())/(train['education_professional.course'].std()))
# train['education_university.degree'] = ((train['education_university.degree'] - train['education_university.degree'].mean())/(train['education_university.degree'].std()))
# train['education_unknown'] = ((train['education_unknown'] - train['education_unknown'].mean())/(train['education_unknown'].std()))
# train['default_no'] = ((train['default_no'] - train['default_no'].mean())/(train['default_no'].std()))
# train['default_unknown'] = ((train['default_unknown'] - train['default_unknown'].mean())/(train['default_unknown'].std()))
# train['default_yes'] = ((train['default_yes'] - train['default_yes'].mean())/(train['default_yes'].std()))
# train['housing_no'] = ((train['housing_no'] - train['housing_no'].mean())/(train['housing_no'].std()))
# train['housing_unknown'] = ((train['housing_unknown'] - train['housing_unknown'].mean())/(train['housing_unknown'].std()))
#
#
#
#

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 62 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   Unnamed: 0                     41188 non-null  int64
 1   age                            41188 non-null  int64
 2   campaign                       41188 non-null  int64
 3   pdays                          41188 non-null  int64
 4   previous                       41188 non-null  int64
 5   no_previous_contact            41188 non-null  int64
 6   not_working                    41188 non-null  int64
 7   job_admin.                     41188 non-null  int64
 8   job_blue-collar                41188 non-null  int64
 9   job_entrepreneur               41188 non-null  int64
 10  job_housemaid                  41188 non-null  int64
 11  job_management                 41188 non-null  int64
 12  job_retired                    41188 non-null  int64
 13  job_self-employe

In [50]:
# train['job_services'] = ((train['job_services'] - train['job_services'].mean())/(train['job_services'].std()))
#
# train['age'] = ((train['age'] - train['age'].mean())/(train['age'].std()))
# train['campaign'] = ((train['campaign'] - train['campaign'].mean())/(train['campaign'].std()))
# train['pdays'] = ((train['pdays'] - train['pdays'].mean())/(train['pdays'].std()))
# train['previous'] = ((train['previous'] - train['previous'].mean())/(train['previous'].std()))
# train['no_previous_contact'] = ((train['no_previous_contact'] - train['no_previous_contact'].mean())/(train['no_previous_contact'].std()))
# train['not_working'] = ((train['not_working'] - train['not_working'].mean())/(train['not_working'].std()))
# train['job_admin'] = ((train['job_admin'] - train['job_admin'].mean())/(train['job_admin'].std()))
# train['job_blue-collar'] = ((train['job_blue-collar'] - train['job_blue-collar'].mean())/(train['job_blue-collar'].std()))
# train['job_entrepreneur'] = ((train['job_entrepreneur'] - train['job_entrepreneur'].mean())/(train['job_entrepreneur'].std()))
# train['job_housemaid'] = ((train['job_housemaid'] - train['job_housemaid'].mean())/(train['job_housemaid'].std()))
# train['job_management'] = ((train['job_management'] - train['job_management'].mean())/(train['job_management'].std()))
# train['job_retired'] = ((train['job_retired'] - train['job_retired'].mean())/(train['job_retired'].std()))
# train['job_self-employed'] = ((train['job_self-employed'] - train['job_self-employed'].mean())/(train['job_self-employed'].std()))

In [51]:
from imblearn.over_sampling import SMOTE

train_data, test_data = np.split(train.sample(frac=1, random_state=1729), [int(0.7 * len(train))])

In [52]:
y_train = train_data['y_yes']
y_test = test_data['y_yes']
X_train = pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)
X_test = pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1)



In [53]:

os = SMOTE(random_state=0)


In [54]:
from sklearn.linear_model import LogisticRegression

classifiers = {
    "Logisitic Regression Classifier": LogisticRegression(n_jobs=3,random_state=0)
}

In [55]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [56]:
from imblearn.over_sampling import SMOTE
train_data = train_data.dropna(axis = 0, how ='any')
test_data = test_data.dropna(axis = 0, how ='any')

y_train = train_data['y_yes']
y_test = test_data['y_yes']
X_train = train_data.drop(['y_yes'], axis=1)
X_test = test_data.drop(['y_yes'], axis=1)


os = SMOTE(random_state=0)

columns = X_train.columns

os_train_data_X, os_train_data_y = os.fit_sample(X_train, y_train)
os_train_data_X = pd.DataFrame(data=os_train_data_X, columns=columns )
os_train_data_y = pd.DataFrame(data=os_train_data_y, columns=['y_yes'])

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=3, n_neighbors=3, p=2,
           weights='uniform')


knn.fit(os_train_data_X, os_train_data_y)

y_score = knn.predict(X_test)

<ipython-input-56-5846ce4b68a2>:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn.fit(os_train_data_X, os_train_data_y)


In [57]:
print("="*125)
print(color.BOLD + color.UNDERLINE + 'KNN Classifier' + color.END, "\n")
print(color.BOLD + "Confusion Matrix: \n" + color.END)
print(confusion_matrix(y_test, y_score), "\n" )
print(classification_report(y_test, y_score), "\n")
acc = round(accuracy_score(y_test, y_score), 3)
rec = round(recall_score(y_test, y_score), 3)
prec = round(precision_score(y_test, y_score), 3)
print(color.BOLD + f'Accuracy: {acc}')
print(f'Recall: {rec}')
print(f'Precision: {prec}' + color.END)
print("="*125, '\n\n')

KNN Classifier 

Confusion Matrix: 

[[8371 2565]
 [ 576  845]] 

              precision    recall  f1-score   support

           0       0.94      0.77      0.84     10936
           1       0.25      0.59      0.35      1421

    accuracy                           0.75     12357
   macro avg       0.59      0.68      0.60     12357
weighted avg       0.86      0.75      0.79     12357
 

Accuracy: 0.746
Recall: 0.595
Precision: 0.248




In [58]:
import joblib

joblib.dump(knn,'product_model.pkl')

xg = joblib.load('product_model.pkl')

model_columns =  list(X_train.columns)

joblib.dump(model_columns, 'product_model_columns.pkl')

['product_model_columns.pkl']